---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [5]:
import pandas as pd
import numpy as np
#import train, test, address,latlons data
train=pd.read_csv('train.csv', encoding = "ISO-8859-1")
test=pd.read_csv('test.csv')
address=pd.read_csv('addresses.csv')
latlons=pd.read_csv('latlons.csv')
#merge train, test with address, latlons,
train=pd.merge(train, address, on='ticket_id',how='left')
train=pd.merge(train, latlons, on='address',how='left')
test=pd.merge(test, address, on='ticket_id',how='left')
test=pd.merge(test, latlons, on='address',how='left')
#forwardfill NA
#train.lat.fillna(method='pad', inplace=True)
#train.lon.fillna(method='pad', inplace=True)
#test.lat.fillna(method='pad', inplace=True)
#test.lon.fillna(method='pad', inplace=True)
#train.violation_street_number.fillna(method='pad', inplace=True)
#train.mailing_address_str_number.fillna(method='pad', inplace=True)
#test.violation_street_number.fillna(method='pad', inplace=True)
#test.mailing_address_str_number.fillna(method='pad', inplace=True)
#extract zip code from address
test['zip'] = test['address'] .str.extract("(\d*\.?\d+)", expand=True)
train['zip'] = train['address'] .str.extract("(\d*\.?\d+)", expand=True)
test['zip']=pd.to_numeric(test['zip'], errors='coerce')
train['zip']=pd.to_numeric(train['zip'], errors='coerce')
#get rid of unwanted part from mailing address street number
test['mailing_address_str_number'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
train['mailing_address_str_number'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
test['violation_street_number'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
train['violation_street_number'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
test['mailing_address_str_number']=pd.to_numeric(test['mailing_address_str_number'], errors='coerce')
train['mailing_address_str_number']=pd.to_numeric(train['mailing_address_str_number'], errors='coerce')
train['mailing_address_str_number']=pd.to_numeric(train['mailing_address_str_number'], errors='coerce')
test['mailing_address_str_number']=pd.to_numeric(test['mailing_address_str_number'], errors='coerce')
test= test.replace(np.nan, 0, regex=True)
train= train.replace(np.nan, 0, regex=True)
#test.columns
#drop Null situation for compliance
train= train[(train['compliance'] == 0) | (train['compliance'] == 1)]
#define features and target
X=train[['fine_amount','lat','lon','zip','mailing_address_str_number','violation_street_number','clean_up_cost','judgment_amount','admin_fee','state_fee']]
y=train['compliance']
X_test=test[['fine_amount','lat','lon','zip','mailing_address_str_number','violation_street_number','clean_up_cost','judgment_amount','admin_fee','state_fee']]
#use one hot encoder for categorical data
#from sklearn.preprocessing import OneHotEncoder
#ohe = OneHotEncoder(sparse=False)
#X = ohe.fit_transform(X)


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, precision_recall_curve,roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler


def blight_model():
    
    # Your code here
    #X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    #scaler = MinMaxScaler()
    #X_scaled = scaler.fit_transform(X)
    #X_test_scaled = scaler.transform(X_test)
    #clf = SVC(kernel='rbf',C=1).fit(X_scaled, y)
    #y_proba= LogisticRegression().fit(X,y).predict_proba(X_test)
    clf = GradientBoostingClassifier(learning_rate = 0.1, random_state = 0)
    #clf = RandomForestClassifier(random_state=0)
    y_proba= clf.fit(X, y).predict_proba(X_test)
    #y_score = clf.fit(X_train, y_train).decision_function(X_test)
    #fpr, tpr, _ = roc_curve(y_test, y_score)
    #roc_auc = auc(fpr, tpr)
    s = pd.Series(y_proba[:,1], index=test['ticket_id'])
    return s

blight_model()

ticket_id
284932    0.066361
285362    0.024254
285361    0.084624
285338    0.077710
285346    0.094324
285345    0.077710
285347    0.080090
285342    0.314381
285530    0.050963
284989    0.040257
285344    0.074515
285343    0.052933
285340    0.053286
285341    0.080090
285349    0.094324
285348    0.077710
284991    0.057377
285532    0.065043
285406    0.028999
285001    0.078746
285006    0.061896
285405    0.024254
285337    0.038789
285496    0.066500
285497    0.085272
285378    0.030382
285589    0.035167
285585    0.061439
285501    0.083534
285581    0.023386
            ...   
376367    0.041902
376366    0.036322
376362    0.035765
376363    0.065967
376365    0.040229
376364    0.034864
376228    0.050543
376265    0.032007
376286    0.338833
376320    0.034290
376314    0.034776
376327    0.446456
376385    0.422423
376435    0.100169
376370    0.446456
376434    0.063723
376459    0.082880
376478    0.017538
376473    0.039517
376484    0.038910
376482    0.044160
37

In [4]:
#len(blight_model())
#test.shape
#blight_model().shape

61001

In [28]:

#clf = GradientBoostingClassifier(random_state = 0).fit(X,y)
#clf = GradientBoostingClassifier(learning_rate =0.1,random_state = 0)
#clf = RandomForestClassifier(random_state=0)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
#y_score = clf.fit(X_train, y_train).decision_function(X_test)
#fpr, tpr, _ = roc_curve(y_test, y_score)
#roc_auc = auc(fpr, tpr)
#roc_auc
#clf = SVC(kernel='rbf')
#X_test=test[['fine_amount','lat','lon','clean_up_cost']]
#grid_clf_auc = GridSearchCV(clf, param_grid = [{'C': [0.01, 0.1, 1], 'gamma': [0.01, 0.1, 1]}],scoring = 'roc_auc')
#grid_clf_auc = GridSearchCV(clf, param_grid = [{'learning_rate': [0.01, 0.1, 1, 10],'max_depth':[3,4,5]}],scoring = 'roc_auc')
#grid_clf_auc.fit(X, y)
#y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test) 
#grid_clf_auc.best_params_

In [ ]:
#roc_auc
#grid_clf_auc.fit(X, y)
#y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test) 
#grid_clf_auc.best_params_

In [11]:
#test.columns
#grid_clf_auc.best_score_

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status', 'address', 'lat',
       'lon'],
      dtype='object')

In [13]:
#test['zip_code']

0        48213
1        48219
2        48219
3        48183
4        48154
5        48154
6        48154
7        48306
8        29054
9        94539
10       48187
11       48187
12       48209
13       48209
14       75225
15       75225
16       63042
17       48217
18       48228
19       48236
20       48236
21       91335
22       29054
23       48193
24       48193
25       95129
26       84065
27       48235
28       48202
29       48238
         ...  
60971    48235
60972    48235
60973    48237
60974    48237
60975    48221
60976    48221
60977    48207
60978    48112
60979    48034
60980    48221
60981    48215
60982    48202
60983    48201
60984    48206
60985    48226
60986    48205
60987    48207
60988    48207
60989    48310
60990    48124
60991    70810
60992    70810
60993    70810
60994    70810
60995    70810
60996    48037
60997    48037
60998    48213
60999    48224
61000    48203
Name: zip_code, dtype: object